# Прогнозирование оттока клиента банка

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком.

Построить модель с предельно большим значением *F1*-меры. Нужно довести метрику до 0.59.

Дополнительно измерить *AUC-ROC*, сравнивайем её значением с *F1*-мерой.


**Признаки**
* **RowNumber** — индекс строки в данных
* **CustomerId** — уникальный идентификатор клиента
* **Surname** — фамилия
* **CreditScore** — кредитный рейтинг
* **Geography** — страна проживания
* **Gender** — пол
* **Age** — возраст
* **Tenure** — сколько лет человек является клиентом банка
* **Balance** — баланс на счёте
* **NumOfProducts** — количество продуктов банка, используемых клиентом
* **HasCrCard** — наличие кредитной карты
* **IsActiveMember** — активность клиента
* **EstimatedSalary** — предполагаемая зарплата

**Целевой признак**
* **Exited** — факт ухода клиента

## Подготовка данных

In [ ]:
# Подключаем библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from IPython.display import display
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.utils import shuffle
from tqdm import tqdm

import warnings
warnings.simplefilter('ignore')


In [ ]:
# Открываем файл
df = pd.read_csv('/datasets/Churn.csv')

In [ ]:
# Ознакомимся с данными
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


В столбце Tenure 909 пропусков. Это достаточномногои мы небудем их удалять. Вместо этого заменим их на случайные числа.


In [ ]:
data = (np.random.randint(0,10,size = df['Tenure'].isna().sum()))
df.loc[df['Tenure'].isna(),'Tenure'] = data

In [ ]:
# Проверим, что в датафрейме отсутствуют пропущенные значения
df.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [ ]:
# Проверим наличие явных дубликатов
df.duplicated().sum()

0

In [ ]:
# Удалим столбцы с лишней информацией
df = df.drop(columns=['RowNumber', 'Surname','CustomerId'])

In [ ]:
df.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,4.952100,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,96.653299,10.487806,2.895038,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [ ]:
# Трансформация названий в стиль snake_case
df.columns = df.columns.str.lower()
df.rename(columns = {'numofproducts':'num_of_products', 'creditscore':'credit_score', 'hascrcard':'has_cr_card', 'isactivemember':'is_active_member', 'estimatedsalary':'estimated_salary'}, inplace = True )
df.head()

,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


#### Выводы

В таблице приведены данные о клиентах банка. Данные адекватные, без дубликатов, однако в столбце Tenure (сколько лет человек является клиентом банка) обнаружены пропуски.

* Названия столбцов были стилизованы как snake_case
* Пропуски были обработаныи заменены на случайные значения
* В таблице присутствовали и были удалены столбцы с лишней информацией: RowNumber (индекс строки в данных), CustomerId (уникальный идентификатор клиента), Surname (фамилия).

## Исследование задачи

In [ ]:
# Извлечем признаки и целевой признак
features = df.drop('exited', axis=1)
target = df.exited

In [ ]:
# Применим one hot encoding, для избежания попадания в "ловушку фиктивных признаков"
features_cat = ['geography', 'gender']
features_num = features.columns.drop(features_cat).to_list()
features = pd.get_dummies(features, columns=features_cat, drop_first=True)
print(features.shape)
features.head()

(10000, 11)


,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,geography_Germany,geography_Spain,gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,1,0


In [ ]:
# Разделим выборку на тренировочную, валидационную и тестовую
features, features_test, target, target_test = train_test_split(features, target, test_size=0.2, random_state=12345)
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)

In [ ]:
# Масштабируем параметры
numeric = ['credit_score', 'age', 'balance', 'estimated_salary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])

features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

In [ ]:
# Рассмотрим баланс классов
print(f'Доля объектов положительного класса: {sum(df["exited"]/len(df))}')
print(f'Доля объектов отрицательного класса: {1-sum(df["exited"]/len(df))}')

Доля объектов положительного класса: 0.20369999999999389
Доля объектов отрицательного класса: 0.7963000000000061


Дисбаланс целевого признака равен 4 к 1.

### Найдем лучшую модель не учитывая диссбаланс классов:

#### Модель - Случайный лес

In [ ]:
%%time

best_params_f1 = []
best_f1 = 0
best_params_roc_auc = []
best_roc_auc = 0.5
for n_estimators in range(30,40):
    for max_depth in range(10,17,2):
        model_rf_clf = RandomForestClassifier(random_state=12345, n_estimators=n_estimators,
                                                          max_depth=max_depth)
        model_rf_clf.fit(features_train, target_train)
        predictions = model_rf_clf.predict(features_valid)
        f1 = f1_score(target_valid, predictions)
        if f1 > best_f1:
            best_params_f1 = [n_estimators, max_depth]
            best_f1 = f1
        probabilities = model_rf_clf.predict_proba(features)
        probabilities_one = probabilities[:, 1]
        roc_auc = roc_auc_score(target, probabilities_one)
        if roc_auc > best_roc_auc:
            best_params_roc_auc = [n_estimators, max_depth]
            best_roc_auc = roc_auc

print('Лучшее значение f1:',best_f1)
print('Лучшая глубина дерева для f1',best_params_f1)
print('Лучшее значение roc_auc:',best_roc_auc)
print('Лучшая глубина дерева для roc_auc',best_params_roc_auc)

Лучшее значение f1: 0.5714285714285714
Лучшая глубина дерева для f1 [36, 16]
Лучшее значение roc_auc: 0.683240068429903
Лучшая глубина дерева для roc_auc [36, 12]
CPU times: user 10.2 s, sys: 49.4 ms, total: 10.2 s
Wall time: 10.3 s


#### Модель - Логистическая регрессия

In [ ]:
%%time

best_params_f1 = []
best_f1 = 0
best_params_roc_auc = []
best_roc_auc = 0.5
cs = range(1, 100)
for C in cs:
    model_lr_clf = LogisticRegression(random_state=12345,
                                      C=C, max_iter=1000)
    model_lr_clf.fit(features_train, target_train)
    predictions = model_lr_clf.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    if f1 > best_f1:
        best_params_f1 = [C]
        best_f1 = f1
    probabilities = model_lr_clf.predict_proba(features)
    probabilities_one = probabilities[:, 1]
    roc_auc = roc_auc_score(target, probabilities_one)
    if roc_auc > best_roc_auc:
        best_params_roc_auc = [C]
        best_roc_auc = roc_auc

print('Лучшее значение f1:',best_f1)
print('Лучший параметр С для f1',best_params_f1)
print('Лучшее значение roc_auc:',best_roc_auc)
print('Лучший параметр С для roc_auc',best_params_roc_auc)

Лучшее значение f1: 0.30451127819548873
Лучший параметр С для f1 [10]
Лучшее значение roc_auc: 0.5003158078908232
Лучший параметр С для roc_auc [1]
CPU times: user 18.3 s, sys: 51.1 s, total: 1min 9s
Wall time: 1min 9s


#### Модель - Дерево решений


In [ ]:
%%time

best_params_f1 = []
best_f1 = 0
best_params_roc_auc = []
best_roc_auc = 0
for max_depth in range(2,10):
    model_dt_clf = DecisionTreeClassifier(random_state=12345, max_depth=max_depth)
    model_dt_clf.fit(features_train, target_train)
    predictions = model_dt_clf.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    if f1 > best_f1:
        best_params_f1 = max_depth
        best_f1 = f1
    probabilities = model_dt_clf.predict_proba(features)
    probabilities_one = probabilities[:, 1]
    roc_auc = roc_auc_score(target, probabilities_one)
    if roc_auc > best_roc_auc:
        best_params_roc_auc =  max_depth
        best_roc_auc = roc_auc

print('Лучшее значение f1:',best_f1)
print('Лучшая глубина дерева для f1',best_params_f1)
print('Лучшее значение roc_auc:',best_roc_auc)
print('Лучшая глубина дерева для roc_auc',best_params_roc_auc)

Лучшее значение f1: 0.5543307086614172
Лучшая глубина дерева для f1 7
Лучшее значение roc_auc: 0.6405587146064794
Лучшая глубина дерева для roc_auc 3
CPU times: user 166 ms, sys: 20.5 ms, total: 187 ms
Wall time: 253 ms


#### Выводы

* Исходные данные были разделены на тестовую, обучающую и валидационные выборки.
* Был обучены модели случайный лес, логистическая регрессия и дерево решений.
* Наилучший результат f1 (0,57) показала модель случайный лес. Параметр roc_auc оказался равен 0,7.

## Борьба с дисбалансом

На предыдущих этапах исследования был выявлен дисбаланс классов. Будем пробовать масштабировать выборки, это должно улучшить показатели f1.

### Увеличение выборки (upsampling)

In [ ]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)

    return features_upsampled, target_upsampled

In [ ]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

###  Уменьшение выборки (downsampling)¶

In [ ]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])

    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)

    return features_downsampled, target_downsampled

In [ ]:
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

### Модель - Случайный лес

#### Используем данные после увеличения выборки

In [ ]:
%%time

best_params_f1 = []
best_f1 = 0
best_params_roc_auc = []
best_roc_auc = 0.5
for n_estimators in range(30,40):
    for max_depth in range(10,17,2):
        model_rf_clf = RandomForestClassifier(random_state=12345, n_estimators=n_estimators,
                                                          max_depth=max_depth)
        model_rf_clf.fit(features_upsampled, target_upsampled)
        predictions = model_rf_clf.predict(features_valid)
        f1 = f1_score(target_valid, predictions)
        if f1 > best_f1:
            best_params_f1 = [n_estimators, max_depth]
            best_f1 = f1
        probabilities = model_rf_clf.predict_proba(features)
        probabilities_one = probabilities[:, 1]
        roc_auc = roc_auc_score(target, probabilities_one)
        if roc_auc > best_roc_auc:
            best_params_roc_auc = [n_estimators, max_depth]
            best_roc_auc = roc_auc

print('Лучшее значение f1:',best_f1)
print('Лучшая глубина дерева для f1',best_params_f1)
print('Лучшее значение roc_auc:',best_roc_auc)
print('Лучшая глубина дерева для roc_auc',best_params_roc_auc)

Лучшее значение f1: 0.5929919137466306
Лучшая глубина дерева для f1 [37, 16]
Лучшее значение roc_auc: 0.667515576551094
Лучшая глубина дерева для roc_auc [38, 12]
CPU times: user 13 s, sys: 56.8 ms, total: 13 s
Wall time: 13.1 s


#### Используем данные после уменьшения выборки

In [ ]:
%%time

best_params_f1 = []
best_f1 = 0
best_params_roc_auc = []
best_roc_auc = 0.5
for n_estimators in range(30,40):
    for max_depth in range(10,17,2):
        model_rf_clf = RandomForestClassifier(random_state=12345, n_estimators=n_estimators,
                                                          max_depth=max_depth)
        model_rf_clf.fit(features_downsampled, target_downsampled)
        predictions = model_rf_clf.predict(features_valid)
        f1 = f1_score(target_valid, predictions)
        if f1 > best_f1:
            best_params_f1 = [n_estimators, max_depth]
            best_f1 = f1
        probabilities = model_rf_clf.predict_proba(features)
        probabilities_one = probabilities[:, 1]
        roc_auc = roc_auc_score(target, probabilities_one)
        if roc_auc > best_roc_auc:
            best_params_roc_auc = [n_estimators, max_depth]
            best_roc_auc = roc_auc

print('Лучшее значение f1:',best_f1)
print('Лучшая глубина дерева для f1',best_params_f1)
print('Лучшее значение roc_auc:',best_roc_auc)
print('Лучшая глубина дерева для roc_auc',best_params_roc_auc)

Лучшее значение f1: 0.5603864734299516
Лучшая глубина дерева для f1 [32, 12]
Лучшее значение roc_auc: 0.7400376170063863
Лучшая глубина дерева для roc_auc [37, 12]
CPU times: user 5.94 s, sys: 29.5 ms, total: 5.97 s
Wall time: 5.98 s


### Модель - Логистическая регрессия

#### Используем данные после увеличения выборки

In [ ]:
%%time

best_params_f1 = []
best_f1 = 0
best_params_roc_auc = []
best_roc_auc = 0.5
cs = range(1, 100)
for C in cs:
    model_lr_clf = LogisticRegression(random_state=12345,
                                      C=C, max_iter=1000)
    model_lr_clf.fit(features_upsampled, target_upsampled)
    predictions = model_lr_clf.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    if f1 > best_f1:
        best_params_f1 = [C]
        best_f1 = f1
    probabilities = model_lr_clf.predict_proba(features)
    probabilities_one = probabilities[:, 1]
    roc_auc = roc_auc_score(target, probabilities_one)
    if roc_auc > best_roc_auc:
        best_params_roc_auc = [C]
        best_roc_auc = roc_auc

print('Лучшее значение f1:',best_f1)
print('Лучший параметр С для f1',best_params_f1)
print('Лучшее значение roc_auc:',best_roc_auc)
print('Лучший параметр С для roc_auc',best_params_roc_auc)

Лучшее значение f1: 0.47493403693931396
Лучший параметр С для f1 [1]
Лучшее значение roc_auc: 0.5003158078908232
Лучший параметр С для roc_auc [1]
CPU times: user 14.1 s, sys: 30.8 s, total: 44.9 s
Wall time: 44.8 s


#### Используем данные после уменьшения выборки

In [ ]:
%%time

best_params_f1 = []
best_f1 = 0
best_params_roc_auc = []
best_roc_auc = 0.5
cs = range(1, 100)
for C in cs:
    model_lr_clf = LogisticRegression(random_state=12345,
                                      C=C, max_iter=1000)
    model_lr_clf.fit(features_downsampled, target_downsampled)
    predictions = model_lr_clf.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    if f1 > best_f1:
        best_params_f1 = [C]
        best_f1 = f1
    probabilities = model_lr_clf.predict_proba(features)
    probabilities_one = probabilities[:, 1]
    roc_auc = roc_auc_score(target, probabilities_one)
    if roc_auc > best_roc_auc:
        best_params_roc_auc = [C]
        best_roc_auc = roc_auc

print('Лучшее значение f1:',best_f1)
print('Лучший параметр С для f1',best_params_f1)
print('Лучшее значение roc_auc:',best_roc_auc)
print('Лучший параметр С для roc_auc',best_params_roc_auc)

Лучшее значение f1: 0.4771126760563381
Лучший параметр С для f1 [1]
Лучшее значение roc_auc: 0.5003158078908232
Лучший параметр С для roc_auc [1]
CPU times: user 8.11 s, sys: 18.1 s, total: 26.2 s
Wall time: 26.2 s


### Модель - Дерево решений

#### Используем данные после уменьшения выборки

In [ ]:
%%time

best_params_f1 = []
best_f1 = 0
best_params_roc_auc = []
best_roc_auc = 0
for max_depth in range(2,10):
    model_dt_clf = DecisionTreeClassifier(random_state=12345, max_depth=max_depth)
    model_dt_clf.fit(features_downsampled, target_downsampled)
    predictions = model_dt_clf.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    if f1 > best_f1:
        best_params_f1 = max_depth
        best_f1 = f1
    probabilities = model_dt_clf.predict_proba(features)
    probabilities_one = probabilities[:, 1]
    roc_auc = roc_auc_score(target, probabilities_one)
    if roc_auc > best_roc_auc:
        best_params_roc_auc =  max_depth
        best_roc_auc = roc_auc

print('Лучшее значение f1:',best_f1)
print('Лучшая глубина дерева для f1',best_params_f1)
print('Лучшее значение roc_auc:',best_roc_auc)
print('Лучшая глубина дерева для roc_auc',best_params_roc_auc)

Лучшее значение f1: 0.5493482309124766
Лучшая глубина дерева для f1 7
Лучшее значение roc_auc: 0.6269332419638605
Лучшая глубина дерева для roc_auc 4
CPU times: user 144 ms, sys: 43.3 ms, total: 187 ms
Wall time: 206 ms


#### Используем данные после увеличения выборки

In [ ]:
%%time

best_params_f1 = []
best_f1 = 0
best_params_roc_auc = []
best_roc_auc = 0
for max_depth in range(2,10):
    model_dt_clf = DecisionTreeClassifier(random_state=12345, max_depth=max_depth)
    model_dt_clf.fit(features_upsampled, target_upsampled)
    predictions = model_dt_clf.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    if f1 > best_f1:
        best_params_f1 = max_depth
        best_f1 = f1
    probabilities = model_dt_clf.predict_proba(features)
    probabilities_one = probabilities[:, 1]
    roc_auc = roc_auc_score(target, probabilities_one)
    if roc_auc > best_roc_auc:
        best_params_roc_auc =  max_depth
        best_roc_auc = roc_auc

print('Лучшее значение f1:',best_f1)
print('Лучшая глубина дерева для f1',best_params_f1)
print('Лучшее значение roc_auc:',best_roc_auc)
print('Лучшая глубина дерева для roc_auc',best_params_roc_auc)

Лучшее значение f1: 0.5587044534412956
Лучшая глубина дерева для f1 6
Лучшее значение roc_auc: 0.6405587146064794
Лучшая глубина дерева для roc_auc 3
CPU times: user 212 ms, sys: 0 ns, total: 212 ms
Wall time: 224 ms


#### Выводы


* Upsampling привел к увеличению параметра f1 для всех моделей.
* Downsampling в данном случае показал себя хуже чем Upsampling, вероятно это связано с тем что мы работаем с маленьким датасетом и искуственное уменьшение его размера приводит к уменьшению точности.
* Наилучшее значение f1 (0.59) было получено для модели случайного леса и использованием upsampling и составило 0.67.

## Тестирование модели

In [ ]:
# Протестируем модель случайного леса с наилучшими параметрами на тестовой выборке
model_rf_clf = RandomForestClassifier(random_state=12345, n_estimators=38,
                                                          max_depth=14)
model_rf_clf.fit(features_upsampled, target_upsampled)
predictions = model_rf_clf.predict(features_test)
f1 = f1_score(target_test, predictions)
probabilities = model_rf_clf.predict_proba(features)
probabilities_one = probabilities[:, 1]
roc_auc = roc_auc_score(target, probabilities_one)

print('Значение f1:',f1)
print('Значение roc_auc:',roc_auc)

Значение f1: 0.6112469437652812
Значение roc_auc: 0.6648518648120608


## **Выводы**

1)
* В таблице приведены данные о клиентах банка. Данные адекватные, без дубликатов, однако в столбце Tenure (сколько лет человек является клиентом банка) обнаружены пропуски.
* Названия столбцов были стилизованы как snake_case
* Пропуски были обработаныи заменены на случайные значения
* В таблице присутствовали и были удалены столбцы с лишней информацией: RowNumber (индекс строки в данных), CustomerId (уникальный идентификатор клиента), Surname (фамилия).

2)
* Исходные данные были разделены на тестовую, обучающую и валидационные выборки.
* Был обучены модели случайный лес, логистическая регрессия и дерево решений.
* Наилучший результат f1 (0,57) показала модель случайный лес. Параметр roc_auc оказался равен 0,54.


3)
* В начальных данных наблюдался значительный дисбаланс. Так только 20% отведов для целевого признака были позитивными. Наблюдалось низкое значение величины F1 на всех моделях.
* Для борьбы с дисбалансом использовали увеличение (upscaling) и уменьшение (downscaling)
* С использованием обработанных данных додели показали лучшие результаты. Наилучшее значение f1 было получено для модели случайного леса и использованием upsampling и составило 0.6.

4)
* Выбранная модель прошла проверку на адекватность. параметр f1 = 0.61